<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [197]:
import pandas as pd
import psycopg2
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [198]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [199]:
import os
from dotenv import load_dotenv

load_dotenv()
DBNAME = os.getenv("DBNAME")
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")
PORT = os.getenv("PORT")

In [ ]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [201]:
# текст запроса
query_3_1 = f'''select count(id)
                from vacancies
                
'''

In [202]:
# результат запроса
df_3_1 = pd.read_sql_query(query_3_1, connection)
df_3_1

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [203]:
# текст запроса
query_3_2 = f'''select count(id)
                from employers                
'''

In [204]:
# результат запроса
df_3_2 = pd.read_sql_query(query_3_2, connection)
df_3_2

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [205]:
# текст запроса
query_3_3 = f'''select count(id)
                from areas               
'''

In [206]:
# результат запроса
df_3_3 = pd.read_sql_query(query_3_3, connection)
df_3_3

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [207]:
# текст запроса
query_3_4 = f'''select count(id)
                from industries             
'''

In [208]:
# результат запроса
df_3_4 = pd.read_sql_query(query_3_4, connection)
df_3_4

,count
0,294


***

В предоставленных данных имеется информация о 49197 вакансий, что относительно мало для локации с населением более 100 млн человек. Также имеется информация о 23501 работодателей, в среднем на каждого работодателя около 2 вакансий. Скорее всего большинство работодателей имеют минимальную активность на платформе, а большая часть откликов приходится на топ наиболее популярных компаний. Кандидаты находятся в 1362 регионах, что говорит о широком охвате географии. Работодатели в базе представлены в 294 сферах деятельности. 
В целом по предварительной оценке можно сказать, что данные широко охватывают рынок труда, но количество вакансий не соответствует разнообразию данных. Связано это, скорее всего, с ограничением по размеру датасета.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [209]:
# текст запроса
query_4_1 = f'''select a.name, count(v.id) as cnt
                from vacancies as v
                inner join areas as a on v.area_id = a.id
                group by a.name
                order by count(v.id) desc              
'''

In [210]:
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1.head()

,name,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [211]:
# текст запроса
query_4_2 = f'''select count(id) as cnt
                from vacancies as v    
                where v.salary_from is not null  or  v.salary_to is not null        
'''

In [212]:
# результат запроса
df_4_2 = pd.read_sql_query(query_4_2, connection)
df_4_2.head()

,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [213]:
# текст запроса
query_4_3 = f'''select avg(v.salary_from), avg(v.salary_to)
                from vacancies as v    
                where v.salary_from is not null  or  v.salary_to is not null        
'''

In [214]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3.head()

,avg,avg
0,71064.657901,110536.741923


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [215]:
# текст запроса
query_4_4 = f'''select v.schedule, v.employment, count(id) as cnt
                from vacancies as v    
                group by v.schedule, v.employment 
                order by cnt desc
'''

In [216]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [217]:
# текст запроса
query_4_5 = f'''select v.experience,  count(id) as cnt
                from vacancies as v    
                group by v.experience
                order by cnt
'''

In [218]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Традиционно большая часть вакансий сосредоточена в наиболее крупных городах, что делает выборку неравномерной по географии. Половина резюме без данных об одной из двух границ ЗП (скорее всего верхней).
Среднее значение верхней вилки ЗП больше на 50%. 
Также видно, что растет популярность удаленой работы, гибкого графика и частичной занятости. В подовляющеем большинсве вакансий достаточно опыта менее 6 лет.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [219]:
# текст запроса
query_5_1 = f'''SELECT e.name, count(v.id) as cnt
                from vacancies as v
                inner join employers as e on v.employer_id = e.id
                group by e.name
                order by cnt desc
                LIMIT 5
                '''

In [220]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
display(df_5_1.iloc[0], df_5_1.iloc[4])

name    Яндекс
cnt       1933
Name: 0, dtype: object

name    Газпром нефть
cnt               331
Name: 4, dtype: object

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [221]:
# текст запроса
query_5_2 = f'''
                SELECT a.name, count(e.id) as cnt_empl, count(v.id) as cnt_vac
                from areas as a
                left join vacancies as v on v.area_id = a.id
                left join employers as e on e.area = a.id
                group by a.name
                having count(v.id) = 0
                order by  cnt_empl desc
                limit 1
                '''

In [222]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2

,name,cnt_empl,cnt_vac
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [223]:
# текст запроса
query_5_3 = f'''SELECT e.name, count(DISTINCT a.id) as cnt
                FROM areas as a
                inner join vacancies as v on v.area_id = a.id
                inner join employers as e on e.id = v.employer_id
                group by e.name
                order by cnt desc
                '''

In [224]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

,name,cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [225]:
# текст запроса
query_5_4 = f'''SELECT count(*)
                FROM employers as e
                left join employers_industries as i on e.id = i.employer_id
                where i.industry_id is null
                '''

In [226]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [227]:
# текст запроса
query_5_5 = f'''SELECT e.name, count(i.industry_id)
                FROM employers as e
                left join employers_industries as i on e.id = i.employer_id
                group by e.name
                having count(i.industry_id) = 4
                order by e.name
                limit 1 offset 2
                '''

In [228]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [229]:
# текст запроса
query_5_6 = f'''SELECT count(*)
                FROM employers as e
                inner join employers_industries as ei on e.id = ei.employer_id
                inner join industries as i on ei.industry_id  = i.id 
                where i.name = 'Разработка программного обеспечения'
                '''

In [230]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [231]:
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2'
headers = {
    'User-Agent': 'Mozilla/5.0'
}
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "html.parser")
tables = soup.find_all('table')

cities_list = []

for index, string in enumerate(str(tables[0]).split(' ')):
    if 'title=' in string and 'Всероссийская' not in string:

        if '>' in str(tables[0]).split(' ')[index+1]:
            cities_list.append(
                (string.split('"')[1]) + ' ' + (str(tables[0]).split(' ')[index+1]).split('"')[0])
        else:
            cities_list.append(string.split('"')[1])

cities_string = "(" + ", ".join(f"'{city}'" for city in cities_list) + ")"

In [232]:
# текст запроса<table class="standard sortable"><tbody><tr><th scope="col">№</th><th scope="col">Город</th><th data-sort-type="number" scope="col">Население, <br/> тыс. чел.<br/><small>  (на 1 января 2025)<sup class="reference" id="cite_ref-jan01_2-0"><a href="#cite_note-jan01-2"><span class="cite-bracket">[</span>2<span class="cite-bracket">]</span></a></sup></small></th><th data-sort-type="number" scope="col">Население, <br/> тыс. чел.<br/><small> (<a href="/wiki/%D0%92%D1%81%D0%B5%D1%80%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%B0%D1%8F_%D0%BF%D0%B5%D1%80%D0%B5%D0%BF%D0%B8%D1%81%D1%8C_%D0%BD%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_(2020%E2%80%942021)" title="Всероссийская перепись населения (2020—2021)">перепись-2021</a>)<sup class="reference" id="cite_ref-ofperep21_3-0"><a href="#cite_note-ofperep21-3"><span class="cite-bracket">[</span>3<span class="cite-bracket">]</span></a></sup></small></th><th data-sort-type="number" scope="col">Прирост<br/> относительно<br/>  переписи-2010</th><th data-sort-type="number" scope="col">Население, <br/> тыс. чел.<br/><small>(<a href="/wiki/%D0%92%D1%81%D0%B5%D1%80%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%B0%D1%8F_%D0%BF%D0%B5%D1%80%D0%B5%D0%BF%D0%B8%D1%81%D1%8C_%D0%BD%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_(2010)" title="Всероссийская перепись населения (2010)">перепись-2010</a>)<sup class="reference" id="cite_ref-GKS_100_4-0"><a href="#cite_note-GKS_100-4"><span class="cite-bracket">[</span>4<span class="cite-bracket">]</span></a></sup></small></th></tr><tr><td style="text-align:center">1</td><td><a href="/wiki/%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0" title="Москва">Москва</a></td><td data-sort-value="13274" style="text-align:right">13 274</td><td data-sort-value="13010" style="text-align:right">13 010</td><td data-sort-value="13.1" style="text-align:right">13,1 %</td><td data-sort-value="11504" style="text-align:right">11 504</td></tr><tr><td style="text-align:center">2</td><td><a href="/wiki/%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3" title="Санкт-Петербург">Санкт-Петербург</a></td><td data-sort-value="5653" style="text-align:right">5 653</td><td data-sort-value="5602" style="text-align:right">5 602</td><td data-sort-value="14.8" style="text-align:right">14,8 %</td><td data-sort-value="4880" style="text-align:right">4 880</td></tr><tr><td style="text-align:center">3</td><td><a href="/wiki/%D0%9D%D0%BE%D0%B2%D0%BE%D1%81%D0%B8%D0%B1%D0%B8%D1%80%D1%81%D0%BA" title="Новосибирск">Новосибирск</a></td><td data-sort-value="1637" style="text-align:right">1 637</td><td data-sort-value="1634" style="text-align:right">1 634</td><td data-sort-value="10.9" style="text-align:right">10,9 %</td><td data-sort-value="1474" style="text-align:right">1 474</td></tr><tr><td style="text-align:center">4</td><td><a href="/wiki/%D0%95%D0%BA%D0%B0%D1%82%D0%B5%D1%80%D0%B8%D0%BD%D0%B1%D1%83%D1%80%D0%B3" title="Екатеринбург">Екатеринбург</a></td><td data-sort-value="1548" style="text-align:right">1 548</td><td data-sort-value="1544" style="text-align:right">1 544</td><td data-sort-value="14.4" style="text-align:right">14,4 %</td><td data-sort-value="1350" style="text-align:right">1 350</td></tr><tr><td style="text-align:center">5</td><td><a href="/wiki/%D0%9A%D0%B0%D0%B7%D0%B0%D0%BD%D1%8C" title="Казань">Казань</a></td><td data-sort-value="1330" style="text-align:right">1 330</td><td data-sort-value="1309" style="text-align:right">1 309</td><td data-sort-value="14.4" style="text-align:right">14,4 %</td><td data-sort-value="1144" style="text-align:right">1 144</td></tr><tr><td style="text-align:center">6</td><td><a href="/wiki/%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D1%8F%D1%80%D1%81%D0%BA" title="Красноярск">Красноярск</a></td><td data-sort-value="1212" style="text-align:right">1 212</td><td data-sort-value="1188" style="text-align:right">1 188</td><td data-sort-value="21.6" style="text-align:right">21,6 %</td><td data-sort-value="977" style="text-align:right">977</td></tr><tr><td style="text-align:center">7</td><td><a href="/wiki/%D0%9D%D0%B8%D0%B6%D0%BD%D0%B8%D0%B9_%D0%9D%D0%BE%D0%B2%D0%B3%D0%BE%D1%80%D0%BE%D0%B4" title="Нижний Новгород">Нижний Новгород</a></td><td data-sort-value="1198" style="text-align:right">1 198</td><td data-sort-value="1226" style="text-align:right">1 226</td><td data-sort-value="0.2" style="text-align:right"><span class="noprint" style="display: none;">0,2</span>−2,0 %</td><td data-sort-value="1251" style="text-align:right">1 251</td></tr><tr><td style="text-align:center">8</td><td><a href="/wiki/%D0%A7%D0%B5%D0%BB%D1%8F%D0%B1%D0%B8%D0%BD%D1%81%D0%BA" title="Челябинск">Челябинск</a></td><td data-sort-value="1177" style="text-align:right">1 177</td><td data-sort-value="1190" style="text-align:right">1 190</td><td data-sort-value="5.3" style="text-align:right">5,3 %</td><td data-sort-value="1130" style="text-align:right">1 130</td></tr><tr><td style="text-align:center">9</td><td><a href="/wiki/%D0%A3%D1%84%D0%B0" title="Уфа">Уфа</a></td><td data-sort-value="1166" style="text-align:right">1 166</td><td data-sort-value="1145" style="text-align:right">1 145</td><td data-sort-value="7.8" style="text-align:right">7,8 %</td><td data-sort-value="1062" style="text-align:right">1 062</td></tr><tr><td style="text-align:center">10</td><td><a href="/wiki/%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D0%B4%D0%B0%D1%80" title="Краснодар">Краснодар</a></td><td data-sort-value="1155" style="text-align:right">1 155</td><td data-sort-value="1099" style="text-align:right">1 099</td><td data-sort-value="43.8" style="text-align:right">43,8 %</td><td data-sort-value="764" style="text-align:right">764</td></tr><tr><td style="text-align:center">11</td><td><a href="/wiki/%D0%A1%D0%B0%D0%BC%D0%B0%D1%80%D0%B0" title="Самара">Самара</a></td><td data-sort-value="1154" style="text-align:right">1 154</td><td data-sort-value="1173" style="text-align:right">1 173</td><td data-sort-value="0.3" style="text-align:right">0,3 %</td><td data-sort-value="1169" style="text-align:right">1 169</td></tr><tr><td style="text-align:center">12</td><td><a href="/wiki/%D0%A0%D0%BE%D1%81%D1%82%D0%BE%D0%B2-%D0%BD%D0%B0-%D0%94%D0%BE%D0%BD%D1%83" title="Ростов-на-Дону">Ростов-на-Дону</a></td><td data-sort-value="1143" style="text-align:right">1 143</td><td data-sort-value="1142" style="text-align:right">1 142</td><td data-sort-value="4.9" style="text-align:right">4,9 %</td><td data-sort-value="1089" style="text-align:right">1 089</td></tr><tr><td style="text-align:center">13</td><td><a href="/wiki/%D0%9E%D0%BC%D1%81%D0%BA" title="Омск">Омск</a></td><td data-sort-value="1101" style="text-align:right">1 101</td><td data-sort-value="1126" style="text-align:right">1 126</td><td data-sort-value="0.1" style="text-align:right"><span class="noprint" style="display: none;">0,1</span>−3,4 %</td><td data-sort-value="1166" style="text-align:right">1 166</td></tr><tr><td style="text-align:center">14</td><td><a href="/wiki/%D0%92%D0%BE%D1%80%D0%BE%D0%BD%D0%B5%D0%B6" title="Воронеж">Воронеж</a></td><td data-sort-value="1042" style="text-align:right">1 042</td><td data-sort-value="1058" style="text-align:right">1 058</td><td data-sort-value="18.9" style="text-align:right">18,9 %</td><td data-sort-value="890" style="text-align:right">890</td></tr><tr><td style="text-align:center">15</td><td><a href="/wiki/%D0%9F%D0%B5%D1%80%D0%BC%D1%8C" title="Пермь">Пермь</a></td><td data-sort-value="1028" style="text-align:right">1 028</td><td data-sort-value="1034" style="text-align:right">1 034</td><td data-sort-value="4.3" style="text-align:right">4,3 %</td><td data-sort-value="991" style="text-align:right">991</td></tr><tr><td style="text-align:center">16</td><td><a href="/wiki/%D0%92%D0%BE%D0%BB%D0%B3%D0%BE%D0%B3%D1%80%D0%B0%D0%B4" title="Волгоград">Волгоград</a></td><td data-sort-value="1012" style="text-align:right">1 012</td><td data-sort-value="1028" style="text-align:right">1 028</td><td data-sort-value="0.7" style="text-align:right">0,7 %</td><td data-sort-value="1021" style="text-align:right">1 021</td></tr></tbody></table>


query_5_7 = f'''SELECT a.name, count(v.id) as cnt
                FROM employers as e 
                inner join vacancies as v on e.id = v.employer_id
                inner join areas as a on a.id = v.area_id
                where e.name = 'Яндекс' and a.name in {cities_string}              
                group by a.name
                
                
                UNION ALL   
                 
                SELECT 'Total' as name, count(v.id) as cnt
                FROM employers as e 
                inner join vacancies as v on e.id = v.employer_id
                inner join areas as a on a.id = v.area_id
                where e.name = 'Яндекс' and a.name in {cities_string}              
                order by cnt
                
                '''

In [233]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

Наибольшее количество вакансий сосредоточено у таких компаний, как Яндекс, Ростелеком, Тинькофф, СБЕР и Газпром нефть. Особенно выделяется Яндекс, сохраняющий лидерские позиции по числу вакансий и охвату регионов, включая столицу и крупные города-миллионники. Ростелеком, Тинькофф и СБЕР демонстрируют схожую географическую представленность, что говорит о масштабности компаний.

В столице и городах-миллионниках наблюдается концентрация крупнейшего числа вакансий, что объясняется высокой экономической активностью и численностью населения в этих регионах. При этом часть вакансий не привязана к конкретному региону, что характерно для IT-компаний и финансового сектора, где возможна удалённая работа или филиальная структура. 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [234]:
query_6_1 = f'''SELECT count(*)
                from vacancies 
                where  lower(name) like '%data%' or lower(name) like '%данн%'
                               
                '''

In [235]:
# результат запроса
df_6_1 = pd.read_sql_query(query_6_1, connection)
df_6_1

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [236]:
# текст запроса
query_6_2 = f'''SELECT count(*)
                FROM vacancies
                WHERE (
                LOWER(name) LIKE '%data scientist%' OR
                LOWER(name) LIKE '%data science%' OR
                LOWER(name) LIKE '%исследователь данных%' OR
                LOWER(name) LIKE '%ml%' OR
                LOWER(name) LIKE '%machine learning%' OR
                LOWER(name) LIKE '%машинн%обучен%'
                )
                AND LOWER(name) NOT LIKE '%html%'
                AND (
                LOWER(name) LIKE '%junior%' OR
                LOWER(experience) LIKE '%нет опыта%' OR
                LOWER(schedule) LIKE '%стажировка%')
                '''

In [237]:
# результат запроса
df_6_2 = pd.read_sql_query(query_6_2, connection)
df_6_2

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [238]:
# текст запроса
query_6_3 = f'''SELECT count(*)
                FROM vacancies
                WHERE (
                LOWER(name) LIKE '%data scientist%' OR
                LOWER(name) LIKE '%data science%' OR
                LOWER(name) LIKE '%исследователь данных%' OR
                LOWER(name) LIKE '%ml%' OR
                LOWER(name) LIKE '%machine learning%' OR
                LOWER(name) LIKE '%машинн%обучен%'
                )
                AND LOWER(name) NOT LIKE '%html%'
                AND (LOWER(key_skills) LIKE '%sql%' or LOWER(key_skills) LIKE '%postgres%')                
                '''

In [239]:
# результат запроса
df_6_3 = pd.read_sql_query(query_6_3, connection)
df_6_3

,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [240]:
# текст запроса
query_6_4 = f'''SELECT count(*)
                FROM vacancies
                WHERE (
                LOWER(name) LIKE '%data scientist%' OR
                LOWER(name) LIKE '%data science%' OR
                LOWER(name) LIKE '%исследователь данных%' OR
                LOWER(name) LIKE '%ml%' OR
                LOWER(name) LIKE '%machine learning%' OR
                LOWER(name) LIKE '%машинн%обучен%'
                )
                AND LOWER(name) NOT LIKE '%html%'
                AND LOWER(key_skills) LIKE '%python%'               
                '''

In [241]:
# результат запроса
df_6_4 = pd.read_sql(query_6_4, connection)
df_6_4

,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [242]:
# текст запроса
query_6_5 = f'''SELECT AVG(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1) 
                FROM vacancies
                WHERE key_skills IS NOT NULL AND key_skills <> ''  AND
                (LOWER(name) LIKE '%data scientist%' OR
                LOWER(name) LIKE '%data science%' OR
                LOWER(name) LIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                LOWER(name) LIKE '%machine learning%' OR
                LOWER(name) LIKE '%машинн%обучен%'
                )
                AND LOWER(name) NOT LIKE '%html%'
                    
                '''

In [243]:
# результат запроса
df_6_5 = pd.read_sql(query_6_5, connection)
df_6_5
# df_6_5.to_excel('fff.xlsx')

,avg
0,6.406032


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [244]:
# текст запроса
query_6_6 = f'''SELECT experience, AVG(
                CASE
                    WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL THEN (salary_from + salary_to) / 2
                    WHEN salary_from IS NOT NULL THEN salary_from ELSE salary_to END) AS avg_salary
                FROM vacancies
                WHERE key_skills IS NOT NULL AND key_skills <> ''  AND
                (LOWER(name) LIKE '%data scientist%' OR
                LOWER(name) LIKE '%data science%' OR
                LOWER(name) LIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                LOWER(name) LIKE '%machine learning%' OR
                LOWER(name) LIKE '%машинн%обучен%'
                )
                AND LOWER(name) NOT LIKE '%html%'
                group by experience               
                    
                '''

In [245]:
# результат запроса
df_6_6 = pd.read_sql(query_6_6, connection)
df_6_6

,experience,avg_salary
0,Более 6 лет,NaN
1,Нет опыта,74642.857143
2,От 1 года до 3 лет,146703.875000
3,От 3 до 6 лет,243114.666667


***

В большинстве случаев Data Scientist (DS)  должен владеть знаниями баз данных и уметь писать запросы на SQL.
Язык Python является наиболее популярным и распространённым в DS и ML, но не единственным.
Data Science — сложная и междисциплинарная специальность, которая в среднем требует владения более чем 6 ключевыми навыками.
Зарплата существенно зависит от опыта специалиста. Средние зарплаты DS стремительно растут с опытом.
Точных данных по специалистам с опытом более 6 лет в предоставленном датасете нет.

# Общий вывод по проекту

дополнительный анализ и визуализация 

In [246]:
# извлекаем данные из БД
query_0_1 = f'''SELECT v.name as vacantion_name, v.schedule, v.experience, v.salary_from, v.salary_to, e.name as employer_name, a.name as area_name         
                FROM vacancies as v
                inner join employers as e on e.id = v.employer_id  
                left join areas as a on a.id = v.area_id
                    
                '''

In [247]:
# конвертируем данные из БД датафрейм
df_0_1 = pd.read_sql_query(query_0_1, connection)


df_0_1.head()

,vacantion_name,schedule,experience,salary_from,salary_to,employer_name,area_name
0,Компьютерный Мастер,Полный день,Нет опыта,64000.0,NaN,Спецремонт,Камышин
1,Системный администратор,Полный день,От 3 до 6 лет,NaN,NaN,Диалог Регионы,Севастополь
2,Lead Java Developer to Poland,Удаленная работа,От 3 до 6 лет,NaN,NaN,DataArt,Алматы
3,Lead Java Developer to Poland,Удаленная работа,От 3 до 6 лет,NaN,NaN,DataArt,Нур-Султан
4,Специалист службы поддержки с техническими зна...,Удаленная работа,Нет опыта,15000.0,NaN,Яндекс,Находка


In [248]:
# минимизируем кол-во категорий в столбце регион по граничному значению в 1% от общего кол-ва вакансий
region_trashhold = 0.01


region_unique = df_0_1['area_name'].value_counts(normalize=True)


df_0_1['area_name_filtered'] = df_0_1['area_name'].apply(








    lambda x: 'other' if region_unique[x] < region_trashhold else x)


display(f"Изначально было {df_0_1['area_name'].nunique()} уникальных значения региона, после отсечки по граничному значению {region_trashhold*100}% редко встречающиеся регионы отнесены к категории other, после операции осталось {df_0_1['area_name_filtered'].nunique()} уникальных значения")

'Изначально было 769 уникальных значения региона, после отсечки по граничному значению 1.0% редко встречающиеся регионы отнесены к категории other, после операции осталось 24 уникальных значения'

визуализация

In [249]:
df_count_regions_1 = df_0_1.groupby(by='area_name_filtered')[['salary_from', 'salary_to']].agg(
    'median').sort_values(by='salary_from', ascending=False)
bar = px.bar(data_frame=df_count_regions_1,
             x=df_count_regions_1.index,
             y=['salary_from', 'salary_to'],
             barmode='group',
             title='Столбчатая диаграмма по количеству вакансий в различных регионах',
             labels={'x': 'Регион', 'y': 'Количество вакансий'}
             )
bar.show()

На графике видно, что около 35% от всего кол-ва вакансий - это небольшие регионы (other) с количеством вакансий менее 1 % в датасете. Остальные 65% - это крупные города, в первую очередь Москва и Санкт-Петербург. Наибольшие уровни ЗП предлагают в Москве, затем со значительным отставанием следует Санкт-Петербург, Новосибирск и другие крупные города. Наименьшие ЗП предлагают в городах Казахстана и Беларуси. Причем в Минске наибольший разрыв между верхней и нижней планкой ЗП.

In [250]:
df_0_1['diff_salary'] = df_0_1['salary_to'] - df_0_1['salary_from']
mask = df_0_1['diff_salary'] < 300000
hist = px.histogram(data_frame=df_0_1[mask],
                    x='diff_salary', nbins=20,
                    histnorm='probability density',
                    title='Гистограмма разницы между начальным и возможным уровнем ЗП'
                    )
hist.show()

Во многих вакансиях имеются данные как о минимильной ЗП, так и максимально возможной. Распределение разницы между верхней и нижней границами говорит о том, что в болшинсве случаев разница менее 50 тыс.р., но также имеются вакансии с разницей 100 и более тыс.р.

In [251]:
df_count_regions_2 = df_0_1.groupby(by='schedule')[['salary_from', 'salary_to']].agg(
    'median').sort_values(by='salary_from', ascending=False)
bar = px.bar(data_frame=df_count_regions_2, x=df_count_regions_2.index, y=['salary_from', 'salary_to'],
             barmode='group',
             title='Столбчатая диаграмма по медианной нижней планке ЗП в зависимости от формата работы',
             labels={'x': 'Регион', 'y': 'Медианная ЗП (нижняя планка)'}
             )
bar.show()

На диаграмме распределения медианной ЗП в зависимости от формата работы видно, что медианная ЗП при гибком графике и удаленной работе выше, чем при классическом формате работы. Но такой неожиданный на первый взгляд вывод можно объяснить тем, что удаленно и в удобном графике работают высокооплачиваемые специалисты (IT, менеджеры и дт.), а полный рабочий день у большинства профессии, включая низкооплачиваемые. Также на графике видно, что при сменном графике не только наименьший уровень ЗП, но и минимальная разница между нижней и верхней планкой дохода. Вероятно, в таких профессиях нет метрик оценки эффективности сотрудников и инструментов мотивации.

In [252]:
df_count_regions = df_0_1.groupby(by='experience')[['salary_from', 'salary_to']].agg(
    'median').sort_values(by='salary_from', ascending=False)
bar = px.bar(data_frame=df_count_regions, x=df_count_regions.index, y=['salary_from', 'salary_to'],
             barmode='group',
             title='Столбчатая диаграмма по медианной нижней планке ЗП в зависимости от опыта работы',
             labels={'x': 'Регион', 'y': 'Медианная ЗП (нижняя планка)'}
             )
bar.show()

На диаграмме распределения медианной ЗП в зависимости от опыта работы видно, что с ростом опыта растет и предлагаемая ЗП. Причем наиболее резкий рост при достижении опыта более 3 лет, при дальнейшем увеличении опыта рост ЗП уже менее интенсивнвй.

Общий вывод - наибольшее количесво вакансий и наивысший доход в Москве и других круппных городах. За первые три года работы происходит значительный рост ЗП. Рассчитывать на высокий уровень дохода можно и при удаленной работе или гибком графике.

In [253]:
# в конце работы не забываем закрыть соединение
connection.close()